In [ ]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [ ]:
!git clone https://github.com/Nguyenhieu277/flickr8k.git

Cloning into 'flickr8k'...
remote: Enumerating objects: 8094, done.
remote: Total 8094 (delta 0), reused 0 (delta 0), pack-reused 8094 (from 1)
Receiving objects: 100% (8094/8094), 1.03 GiB | 17.31 MiB/s, done.
Updating files: 100% (8092/8092), done.


In [ ]:
from tensorflow.keras.applications import DenseNet201


model = DenseNet201()
densenet = Model(inputs=model.inputs, outputs=model.layers[-2].output)
densenet.summary()

82524592/82524592 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d            │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,408 │ zero_padding2d[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_1          │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 56, 56, 64)     │              0 │ zero_padding2d_1[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 56, 56, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat       │ (None, 56, 56, 96)     │              0 │ pool1[0][0],           │
│ (Concatenate)             │                        │                │ conv2_block1_2_conv[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block2_0_bn    

 Total params: 18,321,984 (69.89 MB)

 Trainable params: 18,092,928 (69.02 MB)

 Non-trainable params: 229,056 (894.75 KB)

In [ ]:
def text_preprocess(text):
    text = text.lower()
    text = text.replace('[^A-Za-z]', '')
    text = text.replace('\s+', ' ')
    text = " ".join([word for word in text.split() if len(text)>1])
    text = '<start> ' + text + ' <end>'
    return text

In [ ]:
import pandas as pd

df = pd.read_csv('/content/flickr8k/captions.txt')

df.head(5)

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [ ]:
df['caption'] = df['caption'].apply(text_preprocess)
captions = df['caption'].to_list()
df.head(5)

,image,caption
0,1000268201_693b08cb0e.jpg,<start> a child in a pink dress is climbing up...
1,1000268201_693b08cb0e.jpg,<start> a girl going into a wooden building . ...
2,1000268201_693b08cb0e.jpg,<start> a little girl climbing into a wooden p...
3,1000268201_693b08cb0e.jpg,<start> a little girl climbing the stairs to h...
4,1000268201_693b08cb0e.jpg,<start> a little girl in a pink dress going in...


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in captions)

images = df['image'].unique().tolist()
n_imgs = len(images)

split_idx = int(0.9 * n_imgs)
train_images = images[:split_idx]
val_images = images[split_idx:]
train = df[df['image'].isin(train_images)]
test = df[df['image'].isin(val_images)]

train.reset_index(inplace=True,drop=True)
test.reset_index(inplace=True,drop=True)

tokenizer.texts_to_sequences([captions[1]])[0]

[3, 1, 19, 316, 64, 1, 196, 117, 2]

In [ ]:
img_size = 224
features = {}
for image in tqdm(df['image'].unique().tolist()):
    img = load_img('/content/flickr8k/Images/' + image, target_size=(img_size,img_size))
    img = img_to_array(img)
    img = img/255.
    img = np.expand_dims(img,axis=0)
    feature = densenet.predict(img, verbose=0)
    features[image] = feature

  0%|          | 0/8091 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


In [ ]:
pickle.dump(features, open('features.pkl', 'wb'))
with open('features.pkl', 'rb') as file:
    features=pickle.load(file)

In [ ]:
def data_generator(df, X_col, y_col, max_length, tokenizer, vocab_size, batch_size, features, shuffle=True):
    n = len(df)
    while True:
        if shuffle:
            df = df.sample(frac=1).reset_index(drop=True)

        for index in range(0, n, batch_size):
            batch = df.iloc[index:index + batch_size]
            X1, X2, y = list(), list(), list()

            images = batch[X_col].tolist()
            for image in images:
                feature = features[image][0]
                captions = batch.loc[batch[X_col] == image, y_col].tolist()

                for caption in captions:
                    seq = tokenizer.texts_to_sequences([caption])[0]

                    padded_seq = pad_sequences([seq], maxlen=max_length, padding='post')[0]

                    for i in range(1, len(padded_seq)):
                        in_seq, out_seq = padded_seq[:i], padded_seq[i]

                        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                        X1.append(feature)

                        in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post')[0]
                        X2.append(in_seq)
                        y.append(out_seq)

            X1, X2, y = np.array(X1), np.array(X2), np.array(y)
            yield (X1, X2), y

In [ ]:
input1 = Input(shape=(1920, ), name = 'image')
input2 = Input(shape = (max_length, ), name = 'text')

img_features = Dropout(0.5)(input1)
img_features = Dense(256, activation = 'relu')(img_features)

text_features = Embedding(vocab_size, 256, mask_zero = True)(input2)
text_features = Dropout(0.5)(text_features)
text_features = LSTM(256)(text_features)

decoder = add([img_features, text_features])
decoder = Dense(256, activation = 'relu')(decoder)
decoder = Dropout(0.5)(decoder)
outputs = Dense(vocab_size, activation = 'softmax')(decoder)

model = Model(inputs = [input1, input2], outputs = outputs)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)         │ (None, 40)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ image (InputLayer)        │ (None, 1920)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 40, 256)        │      2,174,464 │ text[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 1920)           │              0 │ image[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 40, 256)        │              0 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_3 (NotEqual)    │ (None, 40)             │              0 │ text[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 256)            │        491,776 │ dropout_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 256)            │        525,312 │ dropout_10[0][0],      │
│                           │                        │                │ not_equal_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 256)            │              0 │ dense_9[0][0],         │
│                           │                        │                │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 256)            │         65,792 │ add_3[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 256)            │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 8494)           │      2,182,958 │ dropout_11[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,440,302 (20.75 MB)

 Trainable params: 5,440,302 (20.75 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_generator = data_generator(train, 'image', 'caption', max_length, tokenizer, vocab_size, 64, features)
val_generator = data_generator(test, 'image', 'caption', max_length, tokenizer, vocab_size, 64, features)

model.fit(train_generator, epochs=10, steps_per_epoch=len(train)//64, validation_data=val_generator, validation_steps=len(test)//64)

Epoch 1/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 256s 444ms/step - loss: 2.0263 - val_loss: 1.1575
Epoch 2/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 261s 461ms/step - loss: 1.1570 - val_loss: 1.0502
Epoch 3/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 262s 461ms/step - loss: 1.0518 - val_loss: 1.0076
Epoch 4/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 245s 431ms/step - loss: 0.9950 - val_loss: 0.9738
Epoch 5/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 244s 430ms/step - loss: 0.9601 - val_loss: 0.9632
Epoch 6/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 262s 462ms/step - loss: 0.9343 - val_loss: 0.9565
Epoch 7/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 262s 462ms/step - loss: 0.9079 - val_loss: 0.9531
Epoch 8/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 262s 462ms/step - loss: 0.8857 - val_loss: 0.9342
Epoch 9/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 247s 435ms/step - loss: 0.8685 - val_loss: 0.9411
Epoch 10/10
568/568 ━━━━━━━━━━━━━━━━━━━━ 246s 434ms/step - loss: 0.8499 - val_loss: 0.9399


In [ ]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

model.save('caption_imgs.keras')